In [ ]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 
import wave
import xml.etree.ElementTree as ET

In [ ]:
rootdir="../PF/pre-data"
backup = "../PF/backup" # place holder for backup data


In [ ]:
"""
PF

step 1: Make new directory and convert the wav file
step 2: Extract information from .fln file
step 3: Cleaning folder

"""

# (a) Make new directory and convert the wav file

if True: 
    i= 0
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if not os.path.exists(rootdir+"/"+Dir+"/temp"): 
            os.makedirs(rootdir+"/"+Dir+"/temp")
        i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".WAV")):
                actual_filename = "pf_"+str(i)
                a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp", actual_filename))# (source,target(half path),name_of_file)
                print(a)
                if(a==256):
                        print("Did not get converted")
            if(file.endswith(".fln")):
                shutil.copyfile(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp/"+file)
                
# (b) Extract information from .fln file
if True: 
    for Dir in natsort.natsorted(os.listdir(rootdir)):        
        fileName=""
        df1  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df2  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df3  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/temp"):            
            if(file.endswith(".wav")):
                fileName = file.split('.')[-2]+"_"
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            if(file.endswith(".fln")):                
                if(file.endswith(".fln") and file.split('_')[-1]=="01.fln" ):
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        point = 0
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_a = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df1 = df1.append(df_a,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "boundary"):
                                point = 1
                                continue    
                if(file.endswith(".fln") and file.split('_')[-1]=="02.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_b = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df2 = df2.append(df_b,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "boundary"):
                                point = 1
                                continue  
                if(file.endswith(".fln") and file.split('_')[-1]=="03.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_c = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df3 = df3.append(df_c,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "boundary"):
                                point = 1
                                continue 
        frames = []
        frames = [df1, df2, df3]
        df = pd.concat(frames)
        df.reset_index(drop=True, inplace=True)
        time1 = []
        time2=  []
        starttime = []
        endtime = []
        string = []
        string_final = []
        for i in range(len(df)):
            x = df.get_values()[i]
            time1.append(float(x[0]))
            time2.append(float(x[1]))
            string.append(x[2])
            starttime.append(time1[-1])
            endtime.append(time2[-1])
            string_final.append(string[-1]) 
        string_final =  [x.upper() for x in string_final] 
        df_final = pd.DataFrame({'col1':starttime, 'col2':endtime, 'col3':string_final})
        df_final = df_final.drop(df_final[df_final.col3 ==""].index)         
        filename_list = []
        for i in range(0, len(df_final)):                    
            filename_list.extend([fileName+str(i)])    
        df_final['col4'] = filename_list
        print(df_final.shape)
        #----------------------------txt file creation-----------------------------------------------------------------   
        with open(rootdir+"/"+Dir+"/temp/"+"pf.trans.txt", "a") as file:
            for i in range(0, len(df_final)):
                file.write(df_final['col4'].iloc[i]+" "+df_final['col3'].iloc[i]+"\n")
        #----------------------------chuncking-----------------------------------------------------------------   
        for file1 in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp/")):
            if(file1.endswith(".wav")):
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/temp/"+file1, "wav")
                for i in range(0, len(df_final)):
                    START = (df_final['col1'].iloc[i] * 1000)
                    END = (df_final['col2'].iloc[i] * 1000)
                    chunk = myaudio[START:END]     
                    chunk_name = df_final['col4'].iloc[i]+".flac" 
                    if not os.path.exists(rootdir+"/"+Dir+"/temp/"+chunk_name):
                        print("chunking the audio", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/temp/"+chunk_name, format="wav")
            
# (c) Cleaning folder
if True:
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            print(file)
            if(file.endswith(".xml") or file.endswith(".WAV") or file.endswith(".fln") or file.endswith(".html")):
                os.remove(rootdir+"/"+Dir+"/"+file)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            print(file)
            if(file.endswith(".fln") or file.endswith(".wav")):
                os.remove(rootdir+"/"+Dir+"/temp/"+file)
            else:
                shutil.move(rootdir+"/"+Dir+"/temp/"+file, rootdir+"/"+Dir+"/"+file)
        os.rmdir(rootdir+"/"+Dir+"/temp/")
